In [1]:
import sys
import numpy as np 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from copy import deepcopy
from ipywidgets import IntProgress
from IPython.display import display
sys.path.append('./ggbobo-machine-scientist')
sys.path.append('./ggbobo-machine-scientist/Prior/')
from mcmc import *
from parallel import *
from fit_prior import read_prior_par

# ========== 3. 生成合成数据（y = x1^2 + log(x2) + 高斯噪声） ==========
np.random.seed(42)
x1 = np.random.uniform(-2, 2, 500)
x2 = np.random.uniform(0.1, 2, 500)  # 避免 log(0) 错误
true_y = x1**2 + np.log(x2)
noise = np.random.normal(0, 0.1, 500)  # 高斯噪声
y_noisy = true_y + noise


In [2]:

# 组织 DataFrame
data = pd.DataFrame({'x1': x1, 'x2': x2, 'y': y_noisy})

# ========== 4. 读取先验参数 ==========
prior_par = read_prior_par('./Prior/final_prior_param_sq.named_equations.nv13.np13.2016-09-01 17_05_57.196882.dat')


FileNotFoundError: [Errno 2] No such file or directory: './Prior/final_prior_param_sq.named_equations.nv13.np13.2016-09-01 17_05_57.196882.dat'

In [ ]:

# ========== 5. 设定 MCMC 参数 ==========
Ts = [1] + [1.04**k for k in range(1, 20)]  # 温度参数
variables = ['x1', 'x2']
parameters = ['a%d' % i for i in range(5)]  # 假设最多 5 个参数

# ========== 6. 初始化 Bayesian 机器科学家 ==========
pms = Parallel(
    Ts,
    variables=variables,
    parameters=parameters,
    x=data[['x1', 'x2']], 
    y=data['y'],
    prior_par=prior_par,
)

# ========== 7. 执行 MCMC 采样（第一阶段，初步优化） ==========
nstep = 100
f = IntProgress(min=0, max=nstep, description='Running:')
display(f)

for i in range(nstep):
    pms.mcmc_step()
    pms.tree_swap()
    f.value += 1

# ========== 8. 继续 MCMC 采样（第二阶段，深入优化） ==========
nstep = 3000
f = IntProgress(min=0, max=nstep, description='Running:')
display(f)

description_lengths, mdl, mdl_model = [], np.inf, None

for i in range(nstep):
    pms.mcmc_step()
    pms.tree_swap()
    
    description_lengths.append(pms.t1.E)
    
    if pms.t1.E < mdl:
        mdl, mdl_model = pms.t1.E, deepcopy(pms.t1)

    f.value += 1

# ========== 9. 输出最优模型 ==========
print('Best model:\t', mdl_model)
print('Desc. length:\t', mdl)

# ========== 10. 绘制 MCMC 过程中的描述长度收敛情况 ==========
plt.figure(figsize=(15, 5))
plt.plot(description_lengths)
plt.xlabel('MCMC step', fontsize=14)
plt.ylabel('Description length', fontsize=14)
plt.title('MDL model: $%s$' % mdl_model.latex())
plt.show()

# ========== 11. 计算并绘制预测值 vs. 真实值 ==========
plt.figure(figsize=(6, 6))
plt.scatter(mdl_model.predict(data[['x1', 'x2']]), data['y'])
plt.plot((-6, 6), (-6, 6), 'r--')  # 对角线
plt.xlabel('MDL model predictions', fontsize=14)
plt.ylabel('Actual values', fontsize=14)
plt.show()
